In [ ]:
import pandas as pd
from io import StringIO
from sqlalchemy import create_engine
import glob
from datetime import datetime
import pytz
import os
from dotenv import load_dotenv

In [ ]:
load_dotenv()

In [ ]:
# 1. Configuración de conexión (datos de Render)
# Puedes obtener la URL externa en el dashboard de Render (External Database URL)
DB_URL_1 = os.getenv('DB_URL_1')
DB_URL_2 = os.getenv('DB_URL_2')

In [ ]:
def crear_y_cargar_datos(df, nombre_tabla):
    
    # 1. CREAR LA TABLA (Usando SQLAlchemy)
    # Creamos el "motor" de conexión
    engine = create_engine(DB_URL_1)
    
    print(f"Creando la estructura de la tabla '{nombre_tabla}'...")
    
    # df.head(0) toma solo la cabecera (sin filas) para crear la estructura vacía.
    # if_exists='replace' borrará la tabla si ya existe y la creará de nuevo.
    # index=False evita que se cree una columna extra para el índice del DataFrame.
    df.head(0).to_sql(nombre_tabla, engine, if_exists='replace', index=False)
    
    print("Tabla creada. Iniciando carga masiva...")

    # 2. CARGAR LOS DATOS (Usando psycopg2 y COPY para velocidad máxima)
    # Reutilizamos la lógica rápida del consejo anterior
    
    # Conexión "cruda" para el copiado rápido
    conn = engine.raw_connection()
    cursor = conn.cursor()
    
    buffer = StringIO()
    df.to_csv(buffer, index=False, header=False)
    buffer.seek(0)
    
    try:
        cursor.copy_expert(f"COPY {nombre_tabla} FROM STDIN WITH CSV", buffer)
        conn.commit()
        print("¡Datos cargados exitosamente!")
    except Exception as e:
        print(f"Error cargando datos: {e}")
        conn.rollback()
    finally:
        cursor.close()
        conn.close()

In [ ]:
list_dir = glob.glob('./catalogs/*.csv')
list_dir.sort()
list_dir_len = len(list_dir)


In [ ]:

index = 1
for dir in list_dir:
    nombre_tabla = dir.split('/')[-1]
    nombre_tabla = nombre_tabla.split('.')[0]
    nombre_tabla = nombre_tabla.lower()
    
    df_datos = pd.read_csv(dir).astype(str)

    crear_y_cargar_datos(df_datos, nombre_tabla)

    print(f'{index}/{list_dir_len}', dir, datetime.now(pytz.timezone('America/Guatemala')))
    index += 1